In [ ]:
# Test cell
additional_parameters = '''{"type": "MONITOR"}'''


## Imports


In [2]:
! pip install reparse

     |████████████████████████████████| 727kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 645kB 11.5MB/s eta 0:00:01
  Created wheel for reparse: filename=reparse-3.0-cp37-none-any.whl size=7690 sha256=a4f7b68930183cfd9377a84c3dd3a4cdf58a14ec12961d07bbde526bca164c84
  Stored in directory: /home/jovyan/.cache/pip/wheels/30/91/24/cb0f9096c8fe9ef9b5f5792325d52f88c22ef6e5926e6f2122
Successfully built reparse


In [4]:
import json
import logging
import os
import fnmatch

import numpy
from functions import functions
import reparse
import csv
import pandas as pd
from IPython.display import display, HTML
import re
from datetime import datetime

In [ ]:
add_param = json.loads(additional_parameters)
print(add_param)

# Input Files

Specify the location for the log fils to analyze.
The notebook **can't** access the application log files directory, therefor you need to copy the log files into the directory you specify here.

To copy the log files to the input directory:

1. In the Admin tab for the Sisense application got to "System Management -> File Managment"
2. Click on the "data" folder.
3. Create a new folder with the name "logs_to_parse".
4. Copy the logs from your logs directory to the new directory by running:

    `ls -ltr <Path to logs> | awk '{print $9}' |  xargs  -I file  kubectl cp -n sisense file <Storage Pod ID>:/opt/sisense/storage/data/logs_to_parse`

    for Example:

    `ls -ltr api-gateway.log* | awk '{print $9}' |  xargs  -I file  kubectl cp -n sisense file storage-7db89977d-bk96f:/opt/sisense/storage/data/logs_to_parse`

In [5]:
log_files_dir = "/opt/sisense/storage/data/logs_to_parse"

Specify the logs to parse. You can use wildcards in the pattern to specifiy multiple files.


In [6]:
log_files_pattern = "api-gateway.log*"

## Output Files

1. In the Admin tab for the Sisense application got to "System Management -> File Managment"
2. Click on the "data" folder.
3. Create a new folder with the name "csv".

In [22]:
output_csv_dir = "/opt/sisense/storage/data/csv/"

In [10]:
parsed_log_entries = []

token_spec = r'(?P<TYPE>MONITOR|ERROR|WARN|INFO|TRACE|DEBUG)'

for f in os.listdir(log_files_dir):

    match = fnmatch.fnmatch(f, log_files_pattern)

    if match:
        print("Parsing " + f)
        with open("{0}/{1}".format(log_files_dir, f), newline='\n') as f:
            # read_data = f.reafd()
            # line_num = 1
            parser = reparse.parser(
                            parser_type=reparse.basic_parser,
                            expressions_yaml_path='expression.yaml',
                            patterns_yaml_path='patterns.yaml',
                            functions=functions
                        )
            raw_entry_buffer = ""

            for line in f:

                mo = re.search(token_spec, line)
                if mo:
                    res = mo.group()
                    # print(res)
                    # if res == add_param["type"]:

                    if raw_entry_buffer == "":

                        raw_entry_buffer += line

                    else:

                        mo = re.search(token_spec, raw_entry_buffer)
                        reaesc = re.compile(r'\x1b[^m]*m')
                        raw_entry_buffer = reaesc.sub('', raw_entry_buffer)
                        if parser(raw_entry_buffer):

                            new_entry = parser(raw_entry_buffer)[0]
                            # print(raw_entry_buffer)
                            # print(new_entry)
                            parsed_log_entries.append(new_entry)
                        raw_entry_buffer = line

                else:
                    raw_entry_buffer += line



Parsing api-gateway.log
Parsing api-gateway.log-20210219-1613733301
Parsing api-gateway.log-20210223-1614078902
Parsing api-gateway.log-20210224-1614181503
Parsing api-gateway.log-20210225-1614284110
Parsing api-gateway.log-20210227-1614394801
Parsing api-gateway.log-20210228-1614503710
Parsing api-gateway.log-20210301-1614623402
Parsing api-gateway.log-20210312-1615544102


In [11]:
# for e in parsed_log_entries:
#     print(e)

In [12]:
data_frames = {
    "INFO": pd.DataFrame(),
    "MONITOR": pd.DataFrame()
}

In [13]:
# Cleanup INFO entries
# info_list = list(filter(lambda x: x['type'] == "INFO", parsed_log_entries ))

In [14]:
# Extract monitor entries
monitor_list = list(filter(lambda x: x['type'] == "MONITOR", parsed_log_entries ))


In [16]:
# df_info = pd.DataFrame(info_list)

df_monitor = pd.DataFrame(monitor_list)

In [ ]:
# For testing
# print(df_info.info())
# display(df_info)
#
# print(df_monitor.info())
# display(df_monitor)

 # Make sure the results are as expected (Optional)
# print(df_info.head())
# print(df_monitor.head())

In [20]:
print(output_csv_dir)

/opt/sisense/storage/data/csv


In [24]:
df_monitor.to_csv('{}/monitor.csv'.format(output_csv_dir), index_label="index")
# df_info.to_csv('{}/info.csv'.format(output_csv_dir), index_label="index")



In [19]:
# For using with custom code

df_results = df_monitor

In [ ]:

def manifest_helper(d):
    manifest_cols = []
    for key, value in d.items():
        # print(key, '->', type(value))
        # print(type(value))
        type_map = {
            str: 18,
            bool: 1,
            int: 8,
            numpy.int64: 0,
            float: 6,
            numpy.float64: 6

        }

        if type(value) in type_map.keys():

            c = {
                "type": type_map[type(value)],
                "id": key,
                "name": key
            }


        else:
            c = {
                "type": '',
                "id": key,
                "name": key
            }
        manifest_cols.append(c)
    print(',\n'.join(json.dumps(elem) for elem in manifest_cols))

manifest_helper(df_result.iloc[0])